
### Примерные границы ключевых моментов в процентах:

| Событие                  | Время в минутах | Процент от общей длины (~48 мин) |
|--------------------------|------------------|----------------------------------|
| Cold Open / Teaser       | 0–3              | 0–6%                             |
| Act 1 / Завязка           | 3–12             | 6–25%                            |
| Act 2 / Развитие          | 12–24            | 25–50%                           |
| Midpoint                 | ~24              | 50%                              |
| Act 3 / Подъём к кульминации | 24–36         | 50–75%                           |
| Climax / Кульминация      | 36–42            | 75–87.5%                         |
| Tag / Развязка            | 42–48            | 87.5–100%                        |


In [2]:
total_duration = 2700  # в секундах (длина нашей серии)

In [3]:
# Процентные границы актов
act_boundaries_percent = [
    ('Cold Open / Teaser', 0.00, 0.06),
    ('Act 1 / Завязка', 0.06, 0.25),
    ('Act 2 / Развитие', 0.25, 0.50),
    ('Midpoint', 0.50, 0.50),  # точка
    ('Act 3 / Подъём к кульминации', 0.50, 0.75),
    ('Climax / Кульминация', 0.75, 0.875),
    ('Tag / Развязка', 0.875, 1.00)
]

# Переводим в секунды
act_boundaries_seconds = []
for name, start_p, end_p in act_boundaries_percent:
    start_s = int(total_duration * start_p)
    end_s = int(total_duration * end_p)
    act_boundaries_seconds.append((name, start_s, end_s))

print("Акты в секундах:")
for name, start, end in act_boundaries_seconds:
    print(f"{name}: {start}-{end} сек")

Акты в секундах:
Cold Open / Teaser: 0-162 сек
Act 1 / Завязка: 162-675 сек
Act 2 / Развитие: 675-1350 сек
Midpoint: 1350-1350 сек
Act 3 / Подъём к кульминации: 1350-2025 сек
Climax / Кульминация: 2025-2362 сек
Tag / Развязка: 2362-2700 сек


In [6]:
highlight_length = 20  # секунд (обрезок для рекапа)

highlights = []

# Пример: берем часть из Climax и Tag - тащим из нужных кортежей процентили
climax_start, climax_end = act_boundaries_seconds[4][1], act_boundaries_seconds[4][2] 
tag_start, tag_end = act_boundaries_seconds[5][1], act_boundaries_seconds[5][2]

# Берём последние 20 секунд Climax и первые 20 сек Tag
highlights.append((climax_end - highlight_length, climax_end))
highlights.append((tag_start, tag_start + highlight_length))

# Также можно добавить Midpoint
midpoint = act_boundaries_seconds[3][1]
highlights.append((midpoint - 10, midpoint + 10))

print("\nВыбранные таймкоды для рекапа:")
for start, end in highlights:
    print(f"{start}s - {end}s")


Выбранные таймкоды для рекапа:
2005s - 2025s
2025s - 2045s
1340s - 1360s


> итого минута рекапа

In [7]:
import random

def get_act_boundaries(total_duration=2700, highlight_length = 20, num_highlights = 3):
    '''
    total_duration - длина серии
    highlight_length - базовая длина клипа
    num_highlights - количество клипов для рекапа
    
    '''
    
    # Классичкеские границы в процентилях
    act_boundaries_percent = [
        ('Cold Open / Teaser', 0.00, 0.06),
        ('Act 1 / Завязка', 0.06, 0.25),
        ('Act 2 / Развитие', 0.25, 0.50),
        ('Midpoint', 0.50, 0.50),  # апекс
        ('Act 3 / Подъём к кульминации', 0.50, 0.75),
        ('Climax / Кульминация', 0.75, 0.875),
        ('Tag / Развязка', 0.875, 1.00)
    ]

    # Переводим в секунды
    act_boundaries_seconds = []
    for name, start_p, end_p in act_boundaries_percent:
        start_s = int(total_duration * start_p)
        end_s = int(total_duration * end_p)
        act_boundaries_seconds.append((name, start_s, end_s))
    
    
    highlights = []

    # Пример: берем часть из Climax и Tag - тащим из нужных кортежей процентили
    climax_start, climax_end = act_boundaries_seconds[4][1], act_boundaries_seconds[4][2]
    tag_start, tag_end = act_boundaries_seconds[5][1], act_boundaries_seconds[5][2]
    midpoint = act_boundaries_seconds[3][1]

    # Функция для случайного выбора участка заданной длины внутри промежутка
    def random_clip(start, end, length):
        available_start = max(start, end - length - 1)  # чтобы не выйти за границы
        clip_start = random.randint(available_start, end - 1)
        return (clip_start, clip_start + length)

    # Рандомные клипы в Climax и Tag
    for _ in range(num_highlights // 2):
        highlights.append(random_clip(climax_start, climax_end, highlight_length))
        highlights.append(random_clip(tag_start, tag_end, highlight_length))

    # Midpoint - короткий момент (~20 секунд вокруг точки)
    mp_clip_length = 20
    mp_start = midpoint - random.randint(0, mp_clip_length)
    highlights.append((mp_start, mp_start + mp_clip_length))

        
    return highlights

In [12]:
get_act_boundaries()

[(2020, 2040), (2353, 2373), (1336, 1356)]

In [ ]:
import pysrt
import random
import torch
from transformers import pipeline
import matplotlib.pyplot as plt

# =============================
# 1. Загрузка и обработка субтитров
# =============================
def load_subtitles(srt_file):
    subs = pysrt.open(srt_file)
    blocks = [sub.text for sub in subs]
    times = [(sub.start.ordinal / 1000, sub.end.ordinal / 1000) for sub in subs]  # в секундах
    return blocks, times

# =============================
# 2. Анализ эмоций в блоках текста
# =============================
def analyze_emotions(blocks):
    print("Анализ эмоциональности...")
    emotion_classifier = pipeline("text-classification", model="joeddav/distilbert-base-uncased-go-emotions-student")
    emotions = []
    for block in blocks:
        result = emotion_classifier(block)[0]
        emotions.append(result['score'] if result['label'] not in ['neutral', 'calm'] else 0)
    return emotions

# =============================
# 3. Разделение на акты (в процентах от общей длины)
# =============================
def get_act_boundaries(times):
    total_duration = max(t[1] for t in times)

    act_boundaries_percent = [
        ('Cold Open / Teaser', 0.00, 0.06),
        ('Act 1 / Завязка', 0.06, 0.25),
        ('Act 2 / Развитие', 0.25, 0.50),
        ('Midpoint', 0.50, 0.50),  # точка
        ('Act 3 / Подъём к кульминации', 0.50, 0.75),
        ('Climax / Кульминация', 0.75, 0.875),
        ('Tag / Развязка', 0.875, 1.00)
    ]

    act_boundaries_seconds = []
    for name, start_p, end_p in act_boundaries_percent:
        start_s = int(total_duration * start_p)
        end_s = int(total_duration * end_p)
        act_boundaries_seconds.append((name, start_s, end_s))

    return act_boundaries_seconds, total_duration

# =============================
# 4. Выбор самых напряжённых фрагментов
# =============================
def select_highlights(act_boundaries_seconds, times, emotions, num_highlights=3, highlight_length=20):
    highlights = []

    def random_clip(start, end, length):
        clip_start = random.randint(max(0, start), min(end - length, end))
        return (clip_start, clip_start + length)

    # Для каждого акта считаем среднюю эмоциональную интенсивность
    act_scores = []
    for _, a_start, a_end in act_boundaries_seconds:
        indices = [i for i, (t_start, t_end) in enumerate(times)
                   if t_start >= a_start and t_end <= a_end]
        if indices:
            avg_score = sum(emotions[i] for i in indices) / len(indices)
            act_scores.append((a_start, a_end, avg_score))

    # Сортируем по эмоциональной интенсивности
    act_scores.sort(key=lambda x: x[2], reverse=True)

    # Берём самые горячие участки
    used = set()
    attempts = 0
    while len(highlights) < num_highlights and attempts < 10:
        start, end, _ = act_scores[attempts]
        clip_candidate = random_clip(start, end, highlight_length)
        if all(abs(clip_candidate[0] - h[0]) > 30 for h in highlights):  # избегаем пересечений
            highlights.append(clip_candidate)
        attempts += 1

    return highlights

# =============================
# 5. График активности и вывод результатов
# =============================
def plot_activity(times, emotions, highlights):
    timestamps = [t[0] for t in times]
    plt.figure(figsize=(12, 4))
    plt.plot(timestamps, emotions, label="Эмоциональная интенсивность")
    for start, end in highlights:
        plt.axvspan(start, end, color='red', alpha=0.3)
    plt.title("Распределение эмоциональной активности по серии")
    plt.xlabel("Время (сек)")
    plt.ylabel("Интенсивность")
    plt.legend()
    plt.grid(True)
    plt.show()

# =============================
# Основная функция
# =============================
def main():
    srt_file = "episode.srt"  # заменить на путь к вашему файлу
    blocks, times = load_subtitles(srt_file)
    emotions = analyze_emotions(blocks)
    act_boundaries_seconds, duration = get_act_boundaries(times)
    highlights = select_highlights(act_boundaries_seconds, times, emotions)

    print("\nВыбранные таймкоды для рекапа:")
    for start, end in highlights:
        print(f"{int(start // 60)}:{int(start % 60)} – {int(end // 60)}:{int(end % 60)}")

    plot_activity(times, emotions, highlights)

    print("\nКоманда для сборки рекапа (сохраните в clips.txt):")
    with open('clips.txt', 'w') as f:
        for start, end in highlights:
            f.write(f"file '{srt_file.replace('.srt', '.mp4')}'\n")
            f.write(f"inpoint {start}\n")
            f.write(f"outpoint {end}\n\n")
    print('ffmpeg -f concat -safe 0 -i clips.txt -c copy recap.mp4')

if __name__ == "__main__":
    main()